# Using tensorboard for Visualizing the graph

In [6]:
import tensorflow as tf
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import numpy as np

tf.reset_default_graph()

n_epochs = 4000
learning_rate = 0.01

# Import the dataset
boston_data = load_boston()
m, n = boston_data.data.shape

# Loading the dataset
scaler = StandardScaler()
scaled_boston_housing_data = scaler.fit_transform(boston_data.data)
scaled_boston_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_boston_housing_data]

# Creating the directory name
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_dir = "tf_logs/"
logdir = "{}/run-{}".format(root_dir, now)

# Creating nodes of the graph to hold data
X = tf.constant(scaled_boston_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(boston_data.target.reshape(-1,1), dtype = tf.float32, name = "y")
w = tf.Variable(tf.random_uniform([n+1, 1], -1, 1), name = "weights")

# Calculate the error
y_pred = tf.matmul(X, w, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error, name = "mse"))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
training_op = optimizer.minimize(mse)

# Initializing the Variables of the computation graph
init = tf.global_variables_initializer()

# Information to create logs
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# Running the computation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1000):
        if(epoch % 100 == 0):
            summary_str = mse_summary.eval()
            file_writer.add_summary(summary_str, epoch)
            print("Epoch", epoch, "MSE=", mse.eval())
        sess.run(training_op)
    best_weights = w.eval()
    print(best_weights)

Epoch 0 MSE= 565.1634
Epoch 100 MSE= 32.518936
Epoch 200 MSE= 22.996681
Epoch 300 MSE= 22.438234
Epoch 400 MSE= 22.225662
Epoch 500 MSE= 22.109295
Epoch 600 MSE= 22.040947
Epoch 700 MSE= 21.998514
Epoch 800 MSE= 21.970793
Epoch 900 MSE= 21.95185
[[22.53276   ]
 [-0.8738069 ]
 [ 1.0027533 ]
 [-0.05389305]
 [ 0.70959884]
 [-1.9390731 ]
 [ 2.7321177 ]
 [-0.0357967 ]
 [-3.06338   ]
 [ 2.1529143 ]
 [-1.5400434 ]
 [-2.0239532 ]
 [ 0.85923594]
 [-3.7140393 ]]
